# Libraries

In [47]:
import os

import torch
import torch.nn as nn 
import torch.nn.functional as F 

from torch.utils.data import Dataset
from torch.utils.data import DataLoader 

from torchvision import datasets, transforms
from torchvision.io import read_image
import torchvision.utils as vutils

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from PIL import Image 
import imageio
from einops import rearrange 
from tqdm import tqdm 

import wandb

%matplotlib inline

In [48]:
print("CUDA available:", torch.cuda.is_available())
print("torch version:", torch.__version__)


CUDA available: False
torch version: 2.6.0


# CVAE Model

### Creating the CVAE model.


In [49]:
#######################################################
# CVAE IMPLEMENTATION 
#######################################################

class CVAE(nn.Module): 
    def __init__(self, input_dim=38_804, h1_dim=1024, h2_dim=512, h3_dim=256, latent_dim=100): 
        super().__init__()

        # Encoder Layers 
        self.fc1 = nn.Linear(input_dim, h1_dim)
        self.fc2 = nn.Linear(h1_dim, h2_dim)
        self.fc3 = nn.Linear(h2_dim, h3_dim)
        self.fc41 = nn.Linear(h3_dim, latent_dim)  # mu 
        self.fc42 = nn.Linear(h3_dim, latent_dim)  # log var 

        # Decoder Layers 
        self.fc5 = nn.Linear(latent_dim, h1_dim)
        self.fc6 = nn.Linear(h1_dim, h2_dim)
        self.fc7 = nn.Linear(h2_dim, input_dim)
    
    def encode(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        '''
        The encoder portion of the CVAE. 

        Args: 
            x (Tensor): Input data of shape [batch_size, 178 x 218], flattened images.

        Returns:
            Tuple[Tensor, Tensor]: The mean and log-variance of the approximate posterior 
            distribution q(z|x), where z is the latent variable. 
        '''

        h1 = F.relu(self.fc1(x)) 
        h2 = self.fc2(h1)
        h3 = F.relu(self.fc3(h2))
        mu = self.fc41(h3)
        logvar = self.fc42(h3) 
        
        return mu, logvar

    def reparametrize(self, mu: torch.Tensor, logvar: torch.Tensor) -> torch.Tensor:
        '''
        Reparametrization trick standard in CVAE's. 

        Args: 
            mu: The mean of the approximate posterior distribution: q(z|x)
            logvar: The log-variance of the approximate posterior distribution: q(z|x)
        
        Returns: 
            z: The latent variable sampled from q(z|x) using the reparametrization trick.
            z is size [batch_size, latent_dim], in this case, latent_dim=100.
        ''' 
        
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)

        z = mu + std * eps 
        return z 

    def decode(self, z: torch.Tensor) -> torch.Tensor:
        '''
        The decoder portion of CVAE. 
        Args: 
            z: The latent variable. 

        Returns: 
            x_recon: The reconstructed x sampled from the learned distribution of the decoder. 
            In this case, x_recon is a reconstructred picture. 
        '''
        h1 = F.relu(self.fc5(z))
        h2 = F.relu(self.fc6(h1))
        x_recon = F.sigmoid(self.fc7(h2))

        return x_recon
    
    def forward(self, x: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]: 
        """
        The forward pass on the CVAE. 
        """

        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        x_recon = self.decode(z)
        
        return x_recon, mu, logvar

In [50]:
"""
CVAE TESTING CELL 
"""
 
test_x = torch.rand([1, 178 * 218])   # Test "picture"
#print(test_x)
#print("Shape:", test_x.size())

# Test the forward pass.
cvae = CVAE() 
x_recon, mu, logvar = cvae.forward(test_x)
#print("Reconstructed x:", x_recon)
#print("Shape of x_recon:", x_recon.size())



# Dataset Class

### Create a custom dataset class compatible with the torch.DataLoader class
### https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

The custom dataset class needs 3 functions: `__init__`, `__len__`, and `__getitem__`. 

Use an image directory, in my case "data/CelebA/img_align_celeba/. 

Use an annotations file, which is the metadata for the image, or the relevant features for the model to know. 

Notes: 
    - transforms perform manipulation on data to make it suitable for machine learning models to train on.

In [90]:
class CelebAImageDataset(Dataset):
    def __init__(self, img_dir, annotations_file, transform=None, target_transform=None):
        '''
        Args:
            annotations_file: The dataset containing all of the relevant labels. 
            img_dir: The path to the images folder. 
            transform: Modifies the feature data.
            target_transform: Modifies the label data.   
        '''
        
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir 
        self.transform = transform 
        self.target_transform = target_transform 

        #self.img_landmarks = pd.read_csv(landmarks_file) for simplicity, i wont use this yet.

    
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx: int) -> tuple[torch.Tensor, pd.Series]: 
        '''
        Retrieves the image and its labels of the corresponding index. 
        '''
        # img_dir in this case is ../data/CelebA/img_align_celeba/img_align_celeba
        # ie. ../data/CelebA/img_align_celeba/img_align_celeba/000001.jpg
        
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0]) 
        image = read_image(img_path)
        labels = self.img_labels.iloc[idx, 1:] # retrieves the vector of labels
        labels = torch.tensor(labels) # converts pandas series obj to tensor obj
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            labels = self.target_transform(labels)

        return image, labels 

In [65]:
"""
Testing the dataset class
"""

annotations_file = "../data/CelebA/list_attr_celeba.csv"
img_dir = "../data/CelebA/img_align_celeba/img_align_celeba"

celeba = CelebAImageDataset(img_dir=img_dir, annotations_file=annotations_file)
image, labels = celeba.__getitem__(0) # retrieve image

#display(image)

In [63]:
"""
A testing function to display images. 
"""

def display(image: torch.Tensor): 
    image_np = image.permute(1, 2, 0).numpy() # convert into numpy array with proper dims
    plt.imshow(image_np)
    plt.axis("off")
    plt.show()

# Dataset Manager Class 
### Responsible for partitioning and creating the dataloader. 

In [72]:
from torch.utils.data import Subset

class CelebADatasetManager:
    def __init__(self, img_dir, annotations_file, transform=None, target_transform=None):
        self.dataset = CelebAImageDataset(
            img_dir=img_dir,
            annotations_file=annotations_file,
            transform=transform,
            target_transform=target_transform
        )

        self.split_values = None

    def zero_one_normalize(self):
        """
        
        """

    def partition(self, split_value: float):
        """
        Partitions the dataset based on the split_value. ie. 0.6 means 60% train, 40% test.
        Args: 
            split_value: The value that splits the dataset.
        Returns: 
            tuple[torch.Subset, torch.Subset]: Returns the train and test dataset. 
        """
        if split_value >= 1 or split_value <= 0:
            assert False, "split_value must be between (0,1)"
        
        train_split = split_value 
        test_split = 1 - split_value

        train_idx = round(len(self.dataset) * train_split)
        test_idx = round(len(self.dataset) * test_split)

        train_dataset = Subset(self.dataset, np.arange(train_idx))
        test_dataset = Subset(self.dataset, np.arange(test_idx))

        return train_dataset, test_dataset
    

    def get_dataloader(self, split_value: float, batch_size: int = 64, shuffle:bool = True):
        """
        Creates train and test dataloaders. 

        Args: 
            split_value: The value to split the dataset. ie split_value=0.6 corresponds to 60% train and 40% test. 
            batch_size: The batch size when doing SGD.
            shuffle: Shuffles the dataset after every epoch. 
        
        Returns: 
            A tuple containing the train and test dataloaders.
        """
        train_dataset, test_dataset = self.partition(split_value)
        
        train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
        test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle= shuffle)
        
        return train_dataloader, test_dataloader




In [94]:
"""
Testing the dataset manager class.
"""

annotations_file = "../data/CelebA/list_attr_celeba.csv"
img_dir = "../data/CelebA/img_align_celeba/img_align_celeba"

ds_manager = CelebADatasetManager(img_dir=img_dir, annotations_file=annotations_file)

train_dl, test_dl = ds_manager.get_dataloader(split_value=0.6)
train_features, train_labels = next(iter(train_dl))

train_labels



/var/folders/89/s23t762151989wcy_1bx9vqc0000gn/T/ipykernel_36615/2372924361.py:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  labels = torch.tensor(labels) # converts pandas series obj to tensor obj


tensor([[ 1, -1, -1,  ..., -1, -1, -1],
        [-1, -1,  1,  ..., -1, -1,  1],
        [-1, -1, -1,  ..., -1, -1, -1],
        ...,
        [-1, -1, -1,  ..., -1, -1, -1],
        [-1,  1, -1,  ..., -1, -1,  1],
        [-1, -1,  1,  ..., -1, -1,  1]])